In [9]:
import pandas as pd
import numpy as np
import math

In [10]:
#MeanImputer
class MeanImputer():
    def __init__(self, copy=True):
        self.copy = copy
    def __is_numpy(self, X):
        #X : pandas.DataFrame или numpy.ndarray
        #Is it numpy or not
        return isinstance(X, np.ndarray)
    def fit(self, X, y=None):
        self._encoder_dict = {}
        is_np = self.__is_numpy(X)
        #reshape from 1D to 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        #amount of columns
        ncols = X.shape[1]
        
        if is_np:
            for col in range(ncols):
                self._encoder_dict[col] = np.nanmean(X[:, col])
        else:
            for col in X.columns:
                self._encoder_dict[col] = X[col].mean()
                
        return self
    
    def transform(self, X):
        if self.copy:
            X = X.copy()
        is_np = self.__is_numpy(X)
        
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        ncols = X.shape[1]
        
        if is_np:
            for col in range(ncols):
                X[:, col] = np.nan_to_num(
                    X[:, col],
                nan=self._encoder_dict[col])
        else:
            for col in X.columns:
                X[col] = np.where(X[col].isnull(),
                                    self._encoder_dict[col],
                                    X[col])
        return X    

In [11]:
toy_train = pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]})
toy_train

,Balance,Age
0,8.3,23.0
1,NaN,29.0
2,10.2,36.0
3,3.1,NaN


In [12]:
toy_test = pd.DataFrame(
{'Balance': [10.4, np.NaN, 22.5, 1.1],
'Age': [13, 19, 66, np.NaN]})
toy_test

,Balance,Age
0,10.4,13.0
1,NaN,19.0
2,22.5,66.0
3,1.1,NaN


In [13]:
#Using mean()
'''
for col in toy_train.columns:
    toy_train[col].fillna(toy_train[col].mean(), inplace=True)
    toy_test[col].fillna(toy_train[col].mean(), inplace=True)
print('обучающий датафрейм')
print(toy_train)
print('')
print('тестовый датафрейм')
print(toy_test)
'''

"\nfor col in toy_train.columns:\n    toy_train[col].fillna(toy_train[col].mean(), inplace=True)\n    toy_test[col].fillna(toy_train[col].mean(), inplace=True)\nprint('обучающий датафрейм')\nprint(toy_train)\nprint('')\nprint('тестовый датафрейм')\nprint(toy_test)\n"

In [14]:
imp = MeanImputer()
imp.fit(toy_train)
toy_train = imp.transform(toy_train)
toy_train

,Balance,Age
0,8.3,23.000000
1,7.2,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [15]:
toy_test = imp.transform(toy_test)
toy_test

,Balance,Age
0,10.4,13.000000
1,7.2,19.000000
2,22.5,66.000000
3,1.1,29.333333


In [16]:
toy_train = pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]})
# создаем экземпляр класса, отключив копирование
imp = MeanImputer(copy=False)
# обучаем модель
imp.fit(toy_train[['Age']])
# применяем модель
toy_train['Age'] = imp.transform(toy_train[['Age']])
toy_train

C:\Users\Андрей Куликов\AppData\Local\Temp\ipykernel_13088\1614832478.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = np.where(X[col].isnull(),


,Balance,Age
0,8.3,23.000000
1,NaN,29.000000
2,10.2,36.000000
3,3.1,29.333333


In [17]:
np_toy_train = np.array(pd.DataFrame(
{'Balance': [8.3, np.NaN, 10.2, 3.1],
'Age': [23, 29, 36, np.NaN]}))
np_toy_train

np_toy_test = np.array(pd.DataFrame(
{'Balance': [10.4, np.NaN, 22.5, 1.1],
'Age': [13, 19, 66, np.NaN]}))
np_toy_test

array([[10.4, 13. ],
       [ nan, 19. ],
       [22.5, 66. ],
       [ 1.1,  nan]])

In [18]:
imp.fit(np_toy_train)
np_toy_train = imp.transform(np_toy_train)
np_toy_train

array([[ 8.3       , 23.        ],
       [ 7.2       , 29.        ],
       [10.2       , 36.        ],
       [ 3.1       , 29.33333333]])

In [19]:
np_toy_test = imp.transform(np_toy_test)
np_toy_test

array([[10.4       , 13.        ],
       [ 7.2       , 19.        ],
       [22.5       , 66.        ],
       [ 1.1       , 29.33333333]])

In [29]:
#KNN Model code;

class KNN_Estimator():
    def _euclidean_distance(self, x1, x2):
        distance = 0
        for i in range(len(x1)):
            distance += pow((x1[i] - x2[i]), 2)
        return math.sqrt(distance)
    
    def _vote(self, neighbor_labels):
        counts = np.bincount(neighbor_labels.astype('int'))
        return counts.argmax()
    
    def __init__(self, k=5, task='classification'):
        self.k = k
        self.task = task
        self.k_nearest_neighbors_ = []
        
    def fit(self, X, y):
        self.X_memorized = X
        self.y_memorized = y
        
    def predict(self, X):
        y_pred = np.empty(X.shape[0])
        if self.task == 'classification':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = self._vote(self.k_nearest_neighbors_[i])
        if self.task == 'regression':
            for i, test_sample in enumerate(X):
                idx = np.argsort([self._euclidean_distance(
                    test_sample, x) for x in self.X_memorized])[:self.k]
                k_nearest_neighbors = np.array(
                    [self.y_memorized[i] for i in idx])
                self.k_nearest_neighbors_.append(k_nearest_neighbors)
                y_pred[i] = np.mean(self.k_nearest_neighbors_[i])
        return y_pred    
    

In [26]:
#Classification
X_trn = np.array([[0.1, 0.2, 0.3],
                                [0.7, 0.5, 0.2],
                                [0.1, 0.2, 0.2],
                                [0.9, 0.7, 3.5],
                                [0.2, 0.4, 1.4],
                                [0.4, 0.1, 0.5]])

y_trn = np.array([1, 0, 1, 0, 0, 1])

X_tst = np.array([[0.1, 0.7, 1.1],
                                [0.5, 0.3, 2.8],
                                [0.1, 0.1, 0.2],
                                [0.9, 0.7, 1.5]])

In [31]:
knn = KNN_Estimator(k=3, task='classification')
knn.fit(X_trn, y_trn)

pred = knn.predict(X_tst)
pred

array([1., 0., 1., 0.])

In [32]:
knn.k_nearest_neighbors_

[array([0, 1, 1]), array([0, 0, 1]), array([1, 1, 1]), array([0, 1, 0])]

In [37]:
y_trn = np.array([1.2, 0.5, 1.4, 2.2, 3.5, 5.9])

In [38]:
knn = KNN_Estimator(k=3, task='regression')
knn.fit(X_trn, y_trn)

In [39]:
pred = knn.predict(X_tst)
pred

array([3.53333333, 3.86666667, 2.83333333, 3.3       ])

In [42]:
knn.k_nearest_neighbors_

[array([3.5, 5.9, 1.2]),
 array([2.2, 3.5, 5.9]),
 array([1.4, 1.2, 5.9]),
 array([3.5, 5.9, 0.5])]

In [57]:
a = [2, 4, 7, 9, 14, 20, 21, 22]
b = [3, 5, 8, 10, 14, 20, 21, 30]

def seven(a,b):
    for i in range(len(a)):
            if a[i] % 7 == 0 and b[i] % 7 == 0:
                print(a[i])
            else:
                pass

In [58]:
seven(a,b)

14
21


In [59]:
def find(lst1, lst2):
    lst1 = [i for i in lst1 if i % 7 == 0]
    lst2 = [i for i in lst2 if i % 7 == 0]
    return set(lst1) & set(lst2)

find(a,b)

{14, 21}

In [62]:
a = ["a", "b", "c", "d", "e", "f"]
b = [1, 0, 9, 3, 2, 0]

In [63]:
def get_sorted(lst1, lst2):
    lst_tmp = [x for x in zip(lst1, lst2)]
    lst_tmp.sort(key=lambda x: x[1])
    return [x[0] for x in lst_tmp]

In [64]:
get_sorted(a,b)

['b', 'f', 'a', 'e', 'd', 'c']

In [66]:
z = zip(a,b)
z = list(z)
z

[('a', 1), ('b', 0), ('c', 9), ('d', 3), ('e', 2), ('f', 0)]